In [1]:
# Import libraries
import pandas as pd
# import dask.dataframe as pd
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_absolute_error

from utilities import data_basic_utility as databasic
from utilities import dataframe_utility as dfutil
import features_utility as featutil

## File Details - Light GBM Regression All cols inc nlp

This is a Candidate for being used in an Ensemble 2. 
Characteristicts:
* Light GBM Regression Algorithm
* All columns, including Review Counts and NLP doc vecs
* Uses the full files outputted from A3_130
* Todo: use optimised parameters for Light GBM Regression


In [2]:
filePrefix = "A3_144_sklinearreg_nlp_tinkering"
baseDataDir = "C:/Development/Data/COSC2670/Assignment3/A3data/"
subrunDir = "subruns/"
writeSubRunFile = False
seed = databasic.get_random_seed()

In [3]:

trainFilePath = baseDataDir + 'train_features_preprocessed.csv'
valiFilePath = baseDataDir + 'vali_features_preprocessed.csv'
testFilePath = baseDataDir + 'test_features_preprocessed.csv'

In [4]:
# RowID  BeerID  ReviewerID  BeerName  BeerType  Label
# df_train = pd.read_csv(baseDataDir + 'train_500k.tsv',sep='\t',
df_train = pd.read_csv(trainFilePath)
df_train.head(10)

,RowID,BeerID,ReviewerID,rating,ReviewerReviewCount,BeerReviewCount,BeerType_Altbier,BeerType_AmericanAdjunctLager,BeerType_AmericanAmberRedAle,BeerType_AmericanAmberRedLager,...,Lemmatized_DocVec_190,Lemmatized_DocVec_191,Lemmatized_DocVec_192,Lemmatized_DocVec_193,Lemmatized_DocVec_194,Lemmatized_DocVec_195,Lemmatized_DocVec_196,Lemmatized_DocVec_197,Lemmatized_DocVec_198,Lemmatized_DocVec_199
0,19,12300,10635,4.0,200,23,0,0,0,0,...,-0.046020,-0.085923,-0.020191,0.028098,0.009554,-0.001070,0.058253,-0.020965,-0.060335,-0.025998
1,21,12300,6547,4.5,10,23,0,0,0,0,...,-0.035202,-0.071197,-0.009042,0.013252,-0.038783,-0.011112,0.017132,0.007957,-0.019318,-0.020068
2,23,12300,9789,4.5,164,23,0,0,0,0,...,-0.008459,-0.038928,-0.028027,0.011146,-0.041214,-0.004981,0.029954,0.013247,-0.027631,-0.020938
3,24,12300,7372,5.0,432,23,0,0,0,0,...,-0.043639,-0.069309,-0.031036,0.009859,-0.034390,-0.008547,0.048554,0.007532,-0.044557,-0.019336
4,25,12300,1302,4.5,500,23,0,0,0,0,...,-0.021395,-0.060889,-0.019478,0.016087,-0.051270,-0.012910,0.025220,-0.000260,-0.032785,-0.020871
5,26,12300,704,4.5,605,23,0,0,0,0,...,-0.025142,-0.064408,-0.026324,0.024486,-0.047075,-0.005287,0.025225,0.001942,-0.038939,-0.025383
6,29,12300,1747,5.0,463,23,0,0,0,0,...,-0.024191,-0.071046,-0.025827,0.004000,-0.050098,-0.008236,-0.010039,-0.001957,-0.046674,-0.017337
7,31,12300,9368,4.5,49,23,0,0,0,0,...,-0.035988,-0.069957,-0.026050,0.023177,-0.032557,-0.010953,0.026482,-0.005330,-0.052173,-0.013001
8,32,12300,2568,4.0,221,23,0,0,0,0,...,-0.031430,-0.074136,-0.017969,0.024147,-0.043018,-0.012190,0.002802,0.004149,-0.046826,-0.021015
9,33,12300,6838,4.0,110,23,0,0,0,0,...,-0.038560,-0.071023,-0.026504,0.015529,-0.042741,0.001625,0.001332,0.009657,-0.038897,-0.023400


In [5]:

# df_vali = pd.read_csv(baseDataDir + 'vali_500k.tsv',sep='\t',
df_vali = pd.read_csv(valiFilePath)
df_vali.head(10)

,RowID,BeerID,ReviewerID,rating,ReviewerReviewCount,BeerReviewCount,BeerType_Altbier,BeerType_AmericanAdjunctLager,BeerType_AmericanAmberRedAle,BeerType_AmericanAmberRedLager,...,Lemmatized_DocVec_190,Lemmatized_DocVec_191,Lemmatized_DocVec_192,Lemmatized_DocVec_193,Lemmatized_DocVec_194,Lemmatized_DocVec_195,Lemmatized_DocVec_196,Lemmatized_DocVec_197,Lemmatized_DocVec_198,Lemmatized_DocVec_199
0,22,12300,2634,4.0,19,9,0,0,0,0,...,-0.074742,-0.168165,-0.010900,0.028198,0.006687,0.014946,-0.005977,-0.011162,-0.020651,-0.055318
1,27,12300,5634,4.5,48,9,0,0,0,0,...,-0.039903,-0.081178,-0.026782,0.028400,-0.016471,-0.015230,0.036627,-0.007582,-0.064121,-0.024100
2,28,12300,3544,4.5,227,9,0,0,0,0,...,-0.025519,-0.059343,-0.011759,0.019818,-0.025224,-0.014813,0.010308,-0.004253,-0.056163,-0.023060
3,40,12300,6521,4.0,81,9,0,0,0,0,...,-0.009961,-0.064353,-0.026132,0.024919,-0.032892,-0.024914,0.003314,0.015528,-0.035941,0.005979
4,43,12300,10177,4.5,58,9,0,0,0,0,...,-0.041421,-0.077400,-0.034871,0.025505,-0.007491,-0.014076,0.050549,-0.000238,-0.050784,-0.019350
5,48,12300,2907,3.5,230,9,0,0,0,0,...,-0.009500,-0.068618,-0.037802,0.037846,-0.036413,-0.014508,0.010346,0.011012,-0.065818,-0.002710
6,49,12300,1532,4.0,185,9,0,0,0,0,...,-0.047784,-0.066319,-0.007743,0.041015,-0.026885,-0.021267,0.010073,-0.002857,-0.035936,-0.031098
7,50,12300,3452,3.5,37,9,0,0,0,0,...,0.008967,-0.063812,-0.033023,0.033007,-0.047081,-0.004220,0.012645,0.013312,-0.040105,-0.013771
8,59,12300,6861,4.0,230,9,0,0,0,0,...,-0.031515,-0.084435,-0.039401,0.041123,-0.044032,-0.007342,0.002211,0.014602,-0.036788,-0.007242
9,85539,1198,2634,4.5,19,531,0,0,0,0,...,-0.051095,-0.167019,0.018095,0.029359,0.023710,0.046872,0.023815,-0.026127,0.001300,-0.057575


In [6]:
# "BrewerID_", "BeerType_", "ABV", "DayofWeek", "DayofMonth", "Month", "Year", "Gender_", "TimeOfDay", "Birthday", "BeerName_", "Lemmatized_"
# Get all the columns
col_names = df_train.columns

idCols = ['RowID','BeerID','ReviewerID']
# feature_cols =  col_names.drop(['RowID','BeerID','ReviewerID','rating' ])
target_col = 'rating'

feature_cols = []
# feature_cols.append("ReviewerReviewCount")
# feature_cols.append("BeerReviewCount")
feature_cols = feature_cols + list(filter(lambda x: x.startswith("BeerName_"), col_names))

print(feature_cols)

['BeerName_DocVec_0', 'BeerName_DocVec_1', 'BeerName_DocVec_2', 'BeerName_DocVec_3', 'BeerName_DocVec_4', 'BeerName_DocVec_5', 'BeerName_DocVec_6', 'BeerName_DocVec_7', 'BeerName_DocVec_8', 'BeerName_DocVec_9', 'BeerName_DocVec_10', 'BeerName_DocVec_11', 'BeerName_DocVec_12', 'BeerName_DocVec_13', 'BeerName_DocVec_14', 'BeerName_DocVec_15', 'BeerName_DocVec_16', 'BeerName_DocVec_17', 'BeerName_DocVec_18', 'BeerName_DocVec_19', 'BeerName_DocVec_20', 'BeerName_DocVec_21', 'BeerName_DocVec_22', 'BeerName_DocVec_23', 'BeerName_DocVec_24', 'BeerName_DocVec_25', 'BeerName_DocVec_26', 'BeerName_DocVec_27', 'BeerName_DocVec_28', 'BeerName_DocVec_29', 'BeerName_DocVec_30', 'BeerName_DocVec_31', 'BeerName_DocVec_32', 'BeerName_DocVec_33', 'BeerName_DocVec_34', 'BeerName_DocVec_35', 'BeerName_DocVec_36', 'BeerName_DocVec_37', 'BeerName_DocVec_38', 'BeerName_DocVec_39', 'BeerName_DocVec_40', 'BeerName_DocVec_41', 'BeerName_DocVec_42', 'BeerName_DocVec_43', 'BeerName_DocVec_44', 'BeerName_DocVec_45

In [7]:

# Create the sub data sets of the features and the target
dfTrainIds = df_train[idCols]
dfTrainFeatures = df_train[feature_cols]
dfTrainTarget = df_train[target_col]

dfValiIds = df_vali[idCols]
dfValiFeatures = df_vali[feature_cols]
dfValiTarget = df_vali[target_col]



In [8]:
print(dfTrainFeatures.shape)
dfTrainFeatures.head()

(746207, 200)


,BeerName_DocVec_0,BeerName_DocVec_1,BeerName_DocVec_2,BeerName_DocVec_3,BeerName_DocVec_4,BeerName_DocVec_5,BeerName_DocVec_6,BeerName_DocVec_7,BeerName_DocVec_8,BeerName_DocVec_9,...,BeerName_DocVec_190,BeerName_DocVec_191,BeerName_DocVec_192,BeerName_DocVec_193,BeerName_DocVec_194,BeerName_DocVec_195,BeerName_DocVec_196,BeerName_DocVec_197,BeerName_DocVec_198,BeerName_DocVec_199
0,-0.016229,0.032953,-0.073031,-0.048437,0.130115,-0.053581,0.028545,-0.003937,0.050725,-0.020351,...,0.052876,-0.036183,-0.051693,-0.068958,-0.00546,0.106602,0.010822,0.025523,-0.145516,-0.091074
1,-0.016229,0.032953,-0.073031,-0.048437,0.130115,-0.053581,0.028545,-0.003937,0.050725,-0.020351,...,0.052876,-0.036183,-0.051693,-0.068958,-0.00546,0.106602,0.010822,0.025523,-0.145516,-0.091074
2,-0.016229,0.032953,-0.073031,-0.048437,0.130115,-0.053581,0.028545,-0.003937,0.050725,-0.020351,...,0.052876,-0.036183,-0.051693,-0.068958,-0.00546,0.106602,0.010822,0.025523,-0.145516,-0.091074
3,-0.016229,0.032953,-0.073031,-0.048437,0.130115,-0.053581,0.028545,-0.003937,0.050725,-0.020351,...,0.052876,-0.036183,-0.051693,-0.068958,-0.00546,0.106602,0.010822,0.025523,-0.145516,-0.091074
4,-0.016229,0.032953,-0.073031,-0.048437,0.130115,-0.053581,0.028545,-0.003937,0.050725,-0.020351,...,0.052876,-0.036183,-0.051693,-0.068958,-0.00546,0.106602,0.010822,0.025523,-0.145516,-0.091074


In [9]:
# all cols best params
# {'learning_rate': 0.09075359977364383, 'num_leaves': 120, 'max_depth': 40, 'n_estimators ': 248, 'min_split_gain': 0.6310082232017945, 
# 'min_child_samples': 35, 'subsample': 0.9466694477903548, 
# 'subsample_freq': 0, 'colsample_bytree': 0.29392263338193186, 'reg_alpha': 0.891904482598078, 'reg_lambda': 0.4521335679885054}.

# 

# No tuning
# model = lgb.LGBMRegressor(objective="regression_l1", metric="mae", random_state=seed)

# Same settings as NLP Beer Name
# model = lgb.LGBMRegressor(objective="regression_l1", metric="mae", random_state=seed
#     ,learning_rate=0.2472480229316372, num_leaves = 125, max_depth = 24, n_estimators = 796
#   )

model = LinearRegression()
model.fit(X=dfTrainFeatures, y=dfTrainTarget)

LGBMRegressor(learning_rate=0.2472480229316372, max_depth=24, metric='mae',
              n_estimators=796, num_leaves=125, objective='regression_l1',
              random_state=411453)

In [10]:
# Use the model to predict against our validation data
test_predicted = model.predict(dfValiFeatures)
test_predicted[0:100]

array([3.94859755, 3.94859755, 3.94859755, 3.94859755, 3.94859755,
       3.94859755, 3.94859755, 3.94859755, 3.94859755, 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 ,
       4.0000613 , 4.0000613 , 4.0000613 , 4.0000613 , 4.00006

In [11]:
# dfPredicted = pd.DataFrame({"Predict": test_predicted})
# dfPredicted['Predict'].hist(bins=10)


In [12]:
mae = mean_absolute_error(dfValiTarget, test_predicted)

print("Average MAE: " + str(mae))
print("analyse_maes.append(" + str(mae) + ")")

Average MAE: 0.4524056055395564
analyse_maes.append(0.4524056055395564)


In [13]:

# print("Run: " + filePrefix)
# # Log of Results
# analyse_maes = []

# print("Average MAE over all tests: " + str(np.mean(analyse_maes)))

# Make sure it's predicting floats
# dfPredicted["Predict"].drop_duplicates()

Write to a subrun file

In [14]:
import pandas
print("Average MAE: " + str(mae))

if writeSubRunFile:
  dfPredicted = pandas.DataFrame( { "RowID" : dfValiIds["RowID"], "Score": test_predicted } )
  dfPredicted.to_csv(subrunDir + filePrefix + "_subrun.csv", index=False)

  print(dfPredicted.shape)
  dfPredicted.sort_values("RowID").head(20)


Average MAE: 0.4524056055395564


# Summary

###  Columns: Beer Name with Linear Reg
*
